In [1]:
from urllib.request import urlretrieve
import os

# from the current `notebook`  directory, go back two levels to the `mast30034-project-1-HairongXu` directory
output_relative_dir = '../../data/'


# check if it exists as it makedir will raise an error if it does exist
if not os.path.exists(output_relative_dir):
    os.makedirs(output_relative_dir)

    
    
# now, for each type of data set we will need, we will create the paths
for target_dir in ('curated', 'raw'): 
    if not os.path.exists(output_relative_dir + target_dir):
        os.makedirs(output_relative_dir + target_dir)

In [2]:
import numpy as np
import pandas as pd
from collections import Counter

In [3]:
# dataset download
url = 'https://www.dhhs.vic.gov.au/moving-annual-rents-suburb-march-quarter-2021-excel'
output_dir = '../../data/raw/Annual_rent.xlsx'
urlretrieve(url, output_dir)

('../../data/raw/Annual_rent.xlsx',
 <http.client.HTTPMessage at 0x7f7e80d1fa00>)

In [4]:
path = '../../data/raw/Annual_rent.xlsx'
data = pd.read_excel(path, sheet_name = 'All properties')
data = data.iloc[1:]
data

,Suburbs by region: Moving annual median rent,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 162,Unnamed: 163,Unnamed: 164,Unnamed: 165,Unnamed: 166,Unnamed: 167,Unnamed: 168,Unnamed: 169,Unnamed: 170,Unnamed: 171
1,NaN,NaN,Mar 2000,NaN,Jun 2000,NaN,Sep 2000,NaN,Dec 2000,NaN,...,Mar 2020,NaN,Jun 2020,NaN,Sep 2020,NaN,Dec 2020,NaN,Mar 2021,NaN
2,NaN,NaN,Count,Median,Count,Median,Count,Median,Count,Median,...,Count,Median,Count,Median,Count,Median,Count,Median,Count,Median
3,Inner Melbourne,Albert Park-Middle Park-West St Kilda,1146,260,1136,260,1177,270,1176,275,...,852,600,826,585,802,560,848,550,880,500
4,NaN,Armadale,729,200,731,200,732,205,732,210,...,753,500,782,500,709,500,729,495,782,450
5,NaN,Carlton North,864,260,814,260,800,265,735,270,...,539,585,536,590,520,590,547,590,567,580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,NaN,Wanagaratta,702,125,667,125,629,130,620,130,...,661,290,641,300,642,300,625,310,589,320
158,NaN,Warragul,384,130,366,135,382,135,364,135,...,525,350,567,350,587,360,599,380,576,373
159,NaN,Warrnambool,1271,130,1235,135,1207,135,1136,135,...,1032,340,976,340,973,340,937,350,861,350
160,NaN,Wodonga,1452,145,1443,145,1470,150,1450,150,...,1384,330,1372,330,1326,330,1284,330,1221,340


In [5]:
# get the dataset from excel formate


header = list(data.iloc[0].dropna()) #grab the first row 
header = [s + ' Median' for s in header] # add median for seperate from count

sub_head = ['Region','Suburb']
sub_head.extend(header)


new_header  = data.iloc[[0]].transpose()
new_header.columns = list('a')

new_header['a'] = new_header['a'].astype(str) + ' count' 

new_header.loc[(new_header['a'] == 'nan count'), 'a'] = sub_head

new_header = new_header.transpose().reset_index()

new_header = new_header.drop(['index'], axis=1)

new_header = new_header.iloc[0]
data = data[2:]

data.columns = new_header

data = data.drop(columns = ['Region'])
data = data.loc[:,~data.columns.duplicated()].copy()



In [7]:
# Since the data is record by multiple suburb, split them to different suburb

data['Suburb'] = data['Suburb'].str.split('-')
data = data.explode('Suburb')
data = data[data.Suburb != 'Group Total']
data.reset_index(drop=True, inplace=True)
data.to_csv('../../data/curated/annual_rent_all.csv') 
data

,Suburb,Mar 2000 count,Mar 2000 Median,Jun 2000 count,Jun 2000 Median,Sep 2000 count,Sep 2000 Median,Dec 2000 count,Dec 2000 Median,Mar 2001 count,...,Mar 2020 count,Mar 2020 Median,Jun 2020 count,Jun 2020 Median,Sep 2020 count,Sep 2020 Median,Dec 2020 count,Dec 2020 Median,Mar 2021 count,Mar 2021 Median
0,Albert Park,1146,260,1136,260,1177,270,1176,275,1206,...,852,600,826,585,802,560,848,550,880,500
1,Middle Park,1146,260,1136,260,1177,270,1176,275,1206,...,852,600,826,585,802,560,848,550,880,500
2,West St Kilda,1146,260,1136,260,1177,270,1176,275,1206,...,852,600,826,585,802,560,848,550,880,500
3,Armadale,729,200,731,200,732,205,732,210,716,...,753,500,782,500,709,500,729,495,782,450
4,Carlton North,864,260,814,260,800,265,735,270,715,...,539,585,536,590,520,590,547,590,567,580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,Traralgon,854,125,822,120,830,125,803,125,793,...,921,300,883,300,874,310,839,320,786,330
211,Wanagaratta,702,125,667,125,629,130,620,130,611,...,661,290,641,300,642,300,625,310,589,320
212,Warragul,384,130,366,135,382,135,364,135,365,...,525,350,567,350,587,360,599,380,576,373
213,Warrnambool,1271,130,1235,135,1207,135,1136,135,1072,...,1032,340,976,340,973,340,937,350,861,350
